<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# NRMS: Neural News Recommendation with Multi-Head Self-Attention
NRMS \[1\] is a neural news recommendation approach with multi-head selfattention. The core of NRMS is a news encoder and a user encoder. In the newsencoder, a multi-head self-attentions is used to learn news representations from news titles by modeling the interactions between words. In the user encoder, we learn representations of users from their browsed news and use multihead self-attention to capture the relatedness between the news. Besides, we apply additive
attention to learn more informative news and user representations by selecting important words and news.

## Properties of NRMS:
- NRMS is a content-based neural news recommendation approach.
- It uses multi-self attention to learn news representations by modeling the iteractions between words and learn user representations by capturing the relationship between user browsed news.
- NRMS uses additive attentions to learn informative news and user representations by selecting important words and news.

## Data format:
For quicker training and evaluaiton, we sample MINDdemo dataset of 5k users from [MIND small dataset](https://msnews.github.io/). The MINDdemo dataset has the same file format as MINDsmall and MINDlarge. If you want to try experiments on MINDsmall and MINDlarge, please change the dowload source. Select the MIND_type parameter from ['large', 'small', 'demo'] to choose dataset.
 
**MINDdemo_train** is used for training, and **MINDdemo_dev** is used for evaluation. Training data and evaluation data are composed of a news file and a behaviors file. You can find more detailed data description in [MIND repo](https://github.com/msnews/msnews.github.io/blob/master/assets/doc/introduction.md)

### news data
This file contains news information including newsid, category, subcatgory, news title, news abstarct, news url and entities in news title, entities in news abstarct.
One simple example: <br>

`N46466	lifestyle	lifestyleroyals	The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By	Shop the notebooks, jackets, and more that the royals can't live without.	https://www.msn.com/en-us/lifestyle/lifestyleroyals/the-brands-queen-elizabeth,-prince-charles,-and-prince-philip-swear-by/ss-AAGH0ET?ocid=chopendata	[{"Label": "Prince Philip, Duke of Edinburgh", "Type": "P", "WikidataId": "Q80976", "Confidence": 1.0, "OccurrenceOffsets": [48], "SurfaceForms": ["Prince Philip"]}, {"Label": "Charles, Prince of Wales", "Type": "P", "WikidataId": "Q43274", "Confidence": 1.0, "OccurrenceOffsets": [28], "SurfaceForms": ["Prince Charles"]}, {"Label": "Elizabeth II", "Type": "P", "WikidataId": "Q9682", "Confidence": 0.97, "OccurrenceOffsets": [11], "SurfaceForms": ["Queen Elizabeth"]}]	[]`
<br>

In general, each line in data file represents information of one piece of news: <br>

`[News ID] [Category] [Subcategory] [News Title] [News Abstrct] [News Url] [Entities in News Title] [Entities in News Abstract] ...`

<br>

We generate a word_dict file to transform words in news title to word indexes, and a embedding matrix is initted from pretrained glove embeddings.

### behaviors data
One simple example: <br>
`1	U82271	11/11/2019 3:28:58 PM	N3130 N11621 N12917 N4574 N12140 N9748	N13390-0 N7180-0 N20785-0 N6937-0 N15776-0 N25810-0 N20820-0 N6885-0 N27294-0 N18835-0 N16945-0 N7410-0 N23967-0 N22679-0 N20532-0 N26651-0 N22078-0 N4098-0 N16473-0 N13841-0 N15660-0 N25787-0 N2315-0 N1615-0 N9087-0 N23880-0 N3600-0 N24479-0 N22882-0 N26308-0 N13594-0 N2220-0 N28356-0 N17083-0 N21415-0 N18671-0 N9440-0 N17759-0 N10861-0 N21830-0 N8064-0 N5675-0 N15037-0 N26154-0 N15368-1 N481-0 N3256-0 N20663-0 N23940-0 N7654-0 N10729-0 N7090-0 N23596-0 N15901-0 N16348-0 N13645-0 N8124-0 N20094-0 N27774-0 N23011-0 N14832-0 N15971-0 N27729-0 N2167-0 N11186-0 N18390-0 N21328-0 N10992-0 N20122-0 N1958-0 N2004-0 N26156-0 N17632-0 N26146-0 N17322-0 N18403-0 N17397-0 N18215-0 N14475-0 N9781-0 N17958-0 N3370-0 N1127-0 N15525-0 N12657-0 N10537-0 N18224-0`
<br>

In general, each line in data file represents one instance of an impression. The format is like: <br>

`[Impression ID] [User ID] [Impression Time] [User Click History] [Impression News]`

<br>

User Click History is the user historical clicked news before Impression Time. Impression News is the displayed news in an impression, which format is:<br>

`[News ID 1]-[label1] ... [News ID n]-[labeln]`

<br>
Label represents whether the news is clicked by the user. All information of news in User Click History and Impression News can be found in news data file.

## Global settings and imports

In [2]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.nrms import NRMSModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))


2023-11-29 13:28:27.072464: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


System version: 3.9.12 (main, Apr  5 2022, 01:53:17) 
[Clang 12.0.0 ]
Tensorflow version: 2.12.0


## Prepare parameters

In [11]:
epochs = 5
seed = 42
batch_size = 32

# Options: demo, small, large
MIND_type = 'demo'

## Download and load data

In [12]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'nrms.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

100%|██████████| 17.0k/17.0k [00:02<00:00, 7.60kKB/s]
100%|██████████| 9.84k/9.84k [00:01<00:00, 6.29kKB/s]
100%|██████████| 95.0k/95.0k [00:06<00:00, 15.2kKB/s]


## Create hyper-parameters

In [14]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          batch_size=batch_size,
                          epochs=epochs,
                          show_step=10)
print(hparams)

HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 20, 'head_dim': 20, 'filter_num': 200, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 5, 'batch_size': 32, 'show_step': 10, 'title_size': 30, 'his_size': 50, 'data_format': 'news', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'word_emb_dim': 300, 'model_type': 'nrms', 'loss': 'cross_entropy_loss', 'wordEmb_file': '/var/folders/sv/970s472x5tz75kxxbpmggty40000gn/T/tmp44t_e121/utils/embedding.npy', 'wordDict_file': '/var/folders/sv/970s472x5tz75kxxbpmggty40000gn/T/tmp44t_e121/utils/word_dict.pkl', 'userDict_file': '/var/folders/sv/970s472x5tz75kxxbpmggty40000gn/T/tmp44t_e121/utils/uid2index.pkl'}


## Train the NRMS model

In [15]:
iterator = MINDIterator

In [16]:
model = NRMSModel(hparams, iterator, seed=seed)

2023-11-29 13:57:22.151481: W tensorflow/c/c_api.cc:300] Operation '{name:'att_layer2_1/W/Assign' id:798 op device:{requested: '', assigned: ''} def:{{{node att_layer2_1/W/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](att_layer2_1/W, att_layer2_1/W/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [17]:
valid_behaviors_file

'/var/folders/sv/970s472x5tz75kxxbpmggty40000gn/T/tmp44t_e121/valid/behaviors.tsv'

In [18]:
print(model.run_eval(valid_news_file, valid_behaviors_file))

0it [00:00, ?it/s]/Users/aishwaryasatwani/recommenders/recommender_project/lib/python3.9/site-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-11-29 13:57:30.780700: W tensorflow/c/c_api.cc:300] Operation '{name:'att_layer2_3/b/Assign' id:2814 op device:{requested: '', assigned: ''} def:{{{node att_layer2_3/b/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](att_layer2_3/b, att_layer2_3/b/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
586it [00:06, 87.21it/s]
0it [00:00, ?it/s]2023-11-29 13:57:37.326249: W tensorflow/c/c_api.cc:300] Operation '{name:'att_layer2

{'group_auc': 0.4792, 'mean_mrr': 0.2059, 'ndcg@5': 0.2045, 'ndcg@10': 0.2701}


In [9]:
%%time
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

0it [00:00, ?it/s]2023-11-26 15:15:30.074502: W tensorflow/c/c_api.cc:300] Operation '{name:'loss/mul' id:2002 op device:{requested: '', assigned: ''} def:{{{node loss/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul/x, loss/activation_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-11-26 15:15:30.183277: W tensorflow/c/c_api.cc:300] Operation '{name:'training/Adam/att_layer2_1/b/v/Assign' id:2807 op device:{requested: '', assigned: ''} def:{{{node training/Adam/att_layer2_1/b/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/att_layer2_1/b/v, training/Adam/att_layer2_1/b/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an er

at epoch 1
train info: logloss loss:1.5137877671758115
eval info: group_auc:0.5785, mean_mrr:0.2436, ndcg@10:0.3298, ndcg@5:0.2578
at epoch 1 , train time: 1449.7 eval time: 118.6


step 1080 , total_loss: 1.4181, data_loss: 1.3567: : 1086it [23:37,  1.31s/it]
586it [00:06, 97.22it/s] 
236it [01:47,  2.20it/s]
7538it [00:00, 18310.20it/s]


at epoch 2
train info: logloss loss:1.4181900829021883
eval info: group_auc:0.6004, mean_mrr:0.2552, ndcg@10:0.346, ndcg@5:0.2711
at epoch 2 , train time: 1417.5 eval time: 117.4


step 1080 , total_loss: 1.3796, data_loss: 1.1591: : 1086it [23:47,  1.31s/it]
586it [00:05, 98.63it/s]
236it [01:46,  2.22it/s]
7538it [00:00, 18000.46it/s]


at epoch 3
train info: logloss loss:1.3794216780372746
eval info: group_auc:0.6083, mean_mrr:0.2652, ndcg@10:0.3567, ndcg@5:0.2854
at epoch 3 , train time: 1427.3 eval time: 116.7


step 1080 , total_loss: 1.3537, data_loss: 1.1905: : 1086it [23:45,  1.31s/it]
586it [00:05, 98.78it/s] 
236it [01:47,  2.20it/s]
7538it [00:00, 17152.78it/s]


at epoch 4
train info: logloss loss:1.3537926673340315
eval info: group_auc:0.6102, mean_mrr:0.2684, ndcg@10:0.3596, ndcg@5:0.287
at epoch 4 , train time: 1425.8 eval time: 117.3


step 1080 , total_loss: 1.3297, data_loss: 1.3614: : 1086it [23:44,  1.31s/it]
586it [00:05, 99.35it/s]
236it [01:46,  2.21it/s]
7538it [00:00, 18618.19it/s]


at epoch 5
train info: logloss loss:1.3298480067244332
eval info: group_auc:0.6136, mean_mrr:0.2698, ndcg@10:0.3617, ndcg@5:0.2901
at epoch 5 , train time: 1424.3 eval time: 116.7
CPU times: user 13h 17min 40s, sys: 13min 15s, total: 13h 30min 55s
Wall time: 2h 8min 51s


In [19]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)


586it [00:06, 96.85it/s]
236it [01:48,  2.18it/s]
7538it [00:00, 15671.23it/s]


{'group_auc': 0.4792, 'mean_mrr': 0.2059, 'ndcg@5': 0.2045, 'ndcg@10': 0.2701}
CPU times: user 9min 33s, sys: 6.29 s, total: 9min 39s
Wall time: 1min 58s


In [20]:
sb.glue("res_syn", res_syn)

## Save the model

In [28]:
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)
model.model.save_weights(os.path.join(model_path, "nrms_ckpt"))

In [30]:
checkpoint_dir = "/var/folders/sv/970s472x5tz75kxxbpmggty40000gn/T/tmp44t_e121/model/"
os.listdir(checkpoint_dir)

['checkpoint', 'nrms_ckpt.index', 'nrms_ckpt.data-00000-of-00001']

In [31]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'/var/folders/sv/970s472x5tz75kxxbpmggty40000gn/T/tmp44t_e121/model/nrms_ckpt'

In [34]:
# Load the previously saved weights
model.model.load_weights(latest)

# Re-evaluate the model
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)

586it [00:06, 96.75it/s]
236it [01:46,  2.21it/s]
7538it [00:00, 16604.83it/s]


{'group_auc': 0.4792, 'mean_mrr': 0.2059, 'ndcg@5': 0.2045, 'ndcg@10': 0.2701}


## Output Prediction File

This code segment is used to generate the prediction.zip file, which is in the same format in [MIND Competition Submission Tutorial](https://competitions.codalab.org/competitions/24122#learn_the_details-submission-guidelines).

Please change the `MIND_type` parameter to `large` if you want to submit your prediction to [MIND Competition](https://msnews.github.io/competition.html).

In [37]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(valid_news_file, valid_behaviors_file)

586it [00:06, 95.77it/s]
236it [01:48,  2.18it/s]
7538it [00:00, 16814.67it/s]


In [38]:
len(group_preds)

7538

In [43]:
valid_news_file

'/var/folders/sv/970s472x5tz75kxxbpmggty40000gn/T/tmp44t_e121/valid/news.tsv'

In [48]:
import pandas as pd
news = pd.read_table(valid_news_file)
beh = pd.read_table(valid_behaviors_file)

In [50]:
beh

,1,U41827,11/15/2019 2:41:03 PM,N15366 N12202 N27489 N19773 N21134 N18191 N6863 N11838 N22940 N11838 N7373 N14672 N20561 N16234 N12038 N25392 N26417 N19248 N7214 N20951 N27993 N21 N16998 N20854 N15119 N12563 N2694 N26376 N6178 N11315 N695 N17348 N16106 N18994 N16620 N19892 N3941 N13178 N17369,N23699-0 N21291-0 N1901-0 N27292-0 N17443-0 N18282-0 N6849-0 N13690-0 N12794-0 N8620-1 N28138-0 N23829-0 N13670-0 N6240-0 N1807-0 N3390-0 N17774-0 N14623-0 N26916-0 N13512-0 N21852-0 N28510-0 N27423-0 N21645-0 N26670-0 N21815-0 N9181-0 N6782-0
0,2,U61881,11/15/2019 10:31:42 AM,N16469 N4202 N4202 N21816 N12992 N24242 N7366 ...,N26916-0 N4641-0 N25522-0 N14893-0 N19035-0 N3...
1,3,U54180,11/15/2019 5:36:17 AM,N22427 N16386 N24242 N4385 N14672 N12242 N1852...,N13528-0 N27689-0 N10879-0 N11662-0 N14409-0 N...
2,4,U41164,11/15/2019 9:13:44 AM,N13065 N5748 N12658 N276 N7395 N16010 N13761 N...,N20150-0 N1807-1 N26916-0 N28138-0 N9576-0 N19...
3,5,U8588,11/15/2019 5:39:04 AM,N6629 N4958 N10917 N27079 N828,N21325-0 N5982-0 N19737-1 N9576-0 N20150-0 N25...
4,6,U73735,11/15/2019 11:16:40 AM,N8549 N9763 N28048 N5221 N24080 N5358 N11073 N...,N9576-0 N20946-0 N12037-0 N21325-0 N7667-0 N18...
...,...,...,...,...,...
7532,7534,U23841,11/15/2019 9:07:10 AM,N14405 N417 N16730 N23945 N15405 N25316 N13678...,N26256-0 N28117-0 N2718-0 N16798-0 N27689-0 N6...
7533,7535,U28014,11/15/2019 2:06:47 PM,N1053 N26708 N26570 N12924 N24356 N11647 N2759...,N26670-0 N12794-0 N3390-0 N17443-0 N27292-0 N2...
7534,7536,U89684,11/15/2019 9:07:58 AM,N25457 N27206 N23987 N26009 N545 N2573 N22908 ...,N17443-0 N16798-0 N24553-0 N26096-0 N15927-0 N...
7535,7537,U92611,11/15/2019 11:52:28 AM,N107 N15735 N11987 N16777 N20030 N26557 N8238 ...,N14850-0 N26647-0 N272-0 N22751-0 N21398-0 N26...


In [14]:
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f: 
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')

7538it [00:00, 60006.42it/s]


In [68]:
(np.argsort(np.argsort(group_preds[0])[::-1]) + 1).tolist()

[15,
 7,
 11,
 5,
 21,
 24,
 17,
 16,
 26,
 13,
 18,
 22,
 12,
 2,
 23,
 28,
 8,
 25,
 19,
 10,
 4,
 20,
 1,
 9,
 6,
 14,
 3,
 27]

In [22]:
data_path

'/var/folders/sv/970s472x5tz75kxxbpmggty40000gn/T/tmpyjb6wdcd'

In [15]:
f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()

In [16]:
a = ""
for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
    impr_index += 1
    pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
    pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
    a += ' '.join([str(impr_index), pred_rank])+ '\n'

7538it [00:00, 57803.29it/s]


In [18]:
a

'1 [9,10,20,14,24,19,22,26,2,5,8,1,25,27,23,18,16,11,12,28,3,4,6,13,15,17,7,21]\n2 [25,8,21,44,37,59,22,34,52,9,41,43,35,58,39,10,18,33,36,2,50,26,46,17,1,5,55,6,56,15,61,12,40,57,4,23,30,31,16,13,3,7,42,45,32,60,19,29,48,54,49,11,47,14,24,27,53,28,51,38,20]\n3 [15,2,10,40,5,27,54,23,24,33,29,13,43,17,35,51,34,37,44,7,12,3,22,48,1,42,21,28,47,46,16,9,11,38,41,45,50,36,25,18,4,26,53,6,32,31,52,49,14,30,39,19,20,8]\n4 [10,9,5,4,3,2,11,13,8,1,7,6,12]\n5 [4,8,2,6,5,7,9,10,3,1]\n6 [14,29,5,31,34,10,30,24,28,6,19,26,36,3,20,12,27,32,11,1,21,4,38,9,2,35,7,8,37,17,18,25,22,13,15,23,16,33]\n7 [22,5,36,19,21,1,8,20,10,34,11,37,14,35,3,2,18,16,25,6,30,24,27,7,23,33,29,4,13,12,15,32,17,9,26,31,28]\n8 [99,55,40,53,11,36,39,89,38,84,69,43,88,3,98,27,91,71,57,66,85,97,51,52,59,19,45,41,5,60,13,31,44,12,37,80,72,26,102,10,63,1,68,103,78,14,8,96,50,94,73,62,67,32,61,93,82,74,25,77,35,33,48,9,28,16,70,46,17,30,23,76,54,6,15,79,75,86,34,92,58,18,20,47,22,90,101,7,56,83,2,87,95,42,4,81,49,65,29,24,64,100,

## Reference
\[1\] Wu et al. "Neural News Recommendation with Multi-Head Self-Attention." in Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing and the 9th International Joint Conference on Natural Language Processing (EMNLP-IJCNLP)<br>
\[2\] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics. https://msnews.github.io/competition.html <br>
\[3\] GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/projects/glove/